In [1]:
from notebook_utils import add_project_path
add_project_path()
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import features as f 
from match_station import coord_merge, stations_list

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

In [11]:
atmos = pd.read_csv('../data_raw/atmos.csv')
crash = pd.read_csv('../data_raw/crash_parts.csv', parse_dates=['datetime'])
geo_data = pd.read_csv('../data_raw/geo_data.csv')
# glonas = pd.read_csv('../data_raw/glonas.csv')
meteo = pd.read_csv('../data_raw/meteo.csv')
repair = pd.read_csv('../data_raw/repair.csv', parse_dates=['datetime'])
tele2 = pd.read_csv('../data_raw/tele2_data.csv')
traffic = pd.read_csv('../data_raw/traffic.csv')
train = pd.read_csv('../data_raw/train.csv', parse_dates=['datetime'])
test = pd.read_csv('../data_raw/test.csv')  #parse_dates=['datetime'] 

/Users/evgenyesingildinov/.local/share/virtualenvs/mchs-is38nFyi/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
train

In [ ]:
#train and test set generation
train = f.add_segment_id(train)
test = f.add_segment_id(test)

train_x = f.compile_train(train, 
                         start_date='2015-01-01 00:00:00',
                         end_date='2020-12-31 00:00:00')
train_x = f.add_long_lat(train_x, geo_data)

meteo = f.preprocess_meteo(meteo)

train_x = f.add_meteo_stations(train_x.head(20), meteo)

In [ ]:
train_x

In [72]:
crash_p = f.preprocess_crash_parts(crash)

In [73]:
crash_p.head()

,datetime,road_id,road_km,avuch_start,avuch_end,remuch_length,avuch_loc,stabchar_type,planactiv_type,planactiv_descr,planactiv_year,crash_type,remuch_length_on_segment
0,2017-01-01,9,124,124.77,125.74,970,2,3,4,Дублирование дорожных знаков 3.20 «Обгон запре...,2018-07-01,0,0.23
1,2017-01-01,9,125,124.77,125.74,970,2,3,4,Дублирование дорожных знаков 3.20 «Обгон запре...,2018-07-01,0,0.74
2,2017-01-01,9,131,131.88,132.77,890,2,3,4,Щит аварийно-опасный участок. В 2018 году буде...,2018-03-01,0,0.12
3,2017-01-01,9,132,131.88,132.77,890,2,3,4,Щит аварийно-опасный участок. В 2018 году буде...,2018-03-01,0,0.77
4,2017-01-01,9,145,145.10,145.80,700,2,1,4,Установлен щит «Аварийно-опасный участок». Изм...,2018-09-01,0,0.70


In [74]:
repair_p = f.preprocess_repair(repair)

In [78]:
meteo

,road_id,station,lat,lon,measure_dt,vsp_1,vsp_1_q,vsp_2,vsp_2_q,vsp_3,...,temp_on_measure,temp_on_measure_q,temp_min,temp_min_q,temp_max,temp_max_q,humidity,humidity_q,pressure,pressure_q
0,9,ALEKSAN,56.4,38.75,2015-01-01 18:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-8.4,Значение элемента достоверно,-9.7,Значение элемента достоверно,-8.4,Значение элемента достоверно,95.0,Значение элемента достоверно,986.2,Значение элемента достоверно
1,9,ALEKSAN,56.4,38.75,2015-01-01 21:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-6.8,Значение элемента достоверно,-8.4,Значение элемента достоверно,-6.4,Значение элемента достоверно,92.0,Значение элемента достоверно,985.0,Значение элемента достоверно
2,9,ALEKSAN,56.4,38.75,2015-01-01 00:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-4.8,Значение элемента достоверно,-7.0,Значение элемента достоверно,-4.8,Значение элемента достоверно,97.0,Значение элемента достоверно,984.8,Значение элемента достоверно
3,9,ALEKSAN,56.4,38.75,2015-01-01 03:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-3.5,Значение элемента достоверно,-4.8,Значение элемента достоверно,-3.5,Значение элемента достоверно,96.0,Значение элемента достоверно,985.3,Значение элемента достоверно
4,9,ALEKSAN,56.4,38.75,2015-01-01 06:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-2.6,Значение элемента достоверно,-3.5,Значение элемента достоверно,-2.6,Значение элемента достоверно,95.0,Значение элемента достоверно,986.1,Значение элемента достоверно
5,9,ALEKSAN,56.4,38.75,2015-01-01 09:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-1.5,Значение элемента достоверно,-2.9,Значение элемента достоверно,-1.5,Значение элемента достоверно,93.0,Значение элемента достоверно,986.5,Значение элемента достоверно
6,9,ALEKSAN,56.4,38.75,2015-01-01 12:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-0.8,Значение элемента достоверно,-1.5,Значение элемента достоверно,-0.7,Значение элемента достоверно,94.0,Значение элемента достоверно,985.9,Значение элемента достоверно
7,9,ALEKSAN,56.4,38.75,2015-01-01 15:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,-0.4,Значение элемента достоверно,-0.8,Значение элемента достоверно,-0.4,Значение элемента достоверно,96.0,Значение элемента достоверно,985.2,Значение элемента достоверно
8,9,ALEKSAN,56.4,38.75,2015-01-02 18:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,0.0,Значение элемента достоверно,-0.4,Значение элемента достоверно,0.0,Значение элемента достоверно,96.0,Значение элемента достоверно,983.9,Значение элемента достоверно
9,9,ALEKSAN,56.4,38.75,2015-01-02 21:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,...,0.2,Значение элемента достоверно,-0.1,Значение элемента достоверно,0.2,Значение элемента достоверно,96.0,Значение элемента достоверно,982.5,Значение элемента достоверно


In [92]:
meteo.loc[meteo['station']=='MOSKBAL', 'lat'] = 55.8 # вручную добавляем координаты для станций 'MOSKBAL' и 'MONCHEG', взятые из других датасетов Росгидромета
meteo.loc[meteo['station']=='MOSKBAL', 'lon'] = 37.5
meteo.loc[meteo['station']=='MONCHEG', 'lat'] = 67.9 
meteo.loc[meteo['station']=='MONCHEG', 'lon'] = 32.9

coord_merge(meteo)
full_stations_list = stations_list(meteo)
full_stations_list

,station,lat_long
0,ALEKSAN,"(56.4, 38.75)"


In [94]:
geo

,road_id,road_km,km_name,lat_geoc,lon_geoc,lat_glonass,lon_glonass
0,9,0,"М-8 Холмогоры, 0-й километр",55.755975,37.613846,56.218809,38.087886
1,9,1,"М-8 Холмогоры, 1-й километр",55.758915,37.620349,56.554579,38.587849
2,9,2,"М-8 Холмогоры, 2-й километр",55.761990,37.633704,56.546578,38.594389
3,9,3,"М-8 Холмогоры, 3-й километр",55.768684,37.641095,NaN,NaN
4,9,4,"М-8 Холмогоры, 4-й километр",55.768529,37.631879,NaN,NaN
5,9,5,"М-8 Холмогоры, 5-й километр",55.779331,37.632927,NaN,NaN
6,9,6,"М-8 Холмогоры, 6-й километр",55.789653,37.634691,NaN,NaN
7,9,7,"М-8 Холмогоры, 7-й километр",55.799603,37.635472,NaN,NaN
8,9,8,"М-8 Холмогоры, 8-й километр",55.810190,37.637356,NaN,NaN
9,9,9,"М-8 Холмогоры, 9-й километр",55.819936,37.640275,NaN,NaN
